In [ ]:
#pip install torch transformers datasets soundfile

In [ ]:
""" 
import os
from pydub import AudioSegment

# Paths to your data
mp3_dir = r"C:\Users\alexx\OneDrive\Escriptori\uni\4th year\Advanced Machine Learning\project\cv-corpus-18.0-delta-2024-06-14-ca\cv-corpus-18.0-delta-2024-06-14\ca\clips"
wav_dir = r"C:\Users\alexx\OneDrive\Escriptori\uni\4th year\Advanced Machine Learning\project\cv-corpus-18.0-delta-2024-06-14-ca\cv-corpus-18.0-delta-2024-06-14\ca\wav"

# Iterate over all mp3 files and convert them to wav
for file_name in os.listdir(mp3_dir):
    if file_name.endswith(".mp3"):
        mp3_path = os.path.join(mp3_dir, file_name)
        wav_path = os.path.join(wav_dir, os.path.splitext(file_name)[0] + ".wav")

        # Load mp3 file
        audio = AudioSegment.from_mp3(mp3_path)
        
        # Export as wav file with the desired sample rate (16kHz)
        audio.export(wav_path, format="wav", parameters=["-ar", "16000"])  # "-ar" specifies sample rate
        
print("Conversion completed!")
""" 


Conversion completed!


In [7]:
import os
import pandas as pd
from datasets import load_dataset, Dataset, Audio
from transformers import Wav2Vec2Processor, HubertForCTC, TrainingArguments, Trainer
import torch

In [15]:

# Load the dataset path
dataset_path = r"C:\Users\alexx\OneDrive\Escriptori\uni\4th year\Advanced Machine Learning\project\cv-corpus-18.0-delta-2024-06-14-ca\cv-corpus-18.0-delta-2024-06-14\ca\wav"

# Load TSV files for Common Voice
validated_tsv_path = r"C:\Users\alexx\OneDrive\Escriptori\uni\4th year\Advanced Machine Learning\project\cv-corpus-18.0-delta-2024-06-14-ca\cv-corpus-18.0-delta-2024-06-14\ca\validated.tsv"

# Change the extension of each file in valida_tsv_path to .wav
validated_df = pd.read_csv(validated_tsv_path, sep='\t')
validated_df['path'] = validated_df['path'].apply(lambda x: os.path.splitext(x)[0] + '.wav')


In [16]:
# Correct paths to point to the full audio file paths
validated_df['path'] = validated_df['path'].apply(lambda x: os.path.join(dataset_path, x))

# Create Hugging Face dataset from the data frame
dataset = Dataset.from_pandas(validated_df)

# Add audio column
dataset = dataset.cast_column("path", Audio())

In [17]:
# Load HuBERT processor and model
model_name = "facebook/hubert-large-ls960-ft"  # You can use other variants of HuBERT as well
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = HubertForCTC.from_pretrained(model_name)

c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights

In [20]:
# Preprocessing function
def preprocess_function(batch):
    # Load audio and prepare input features
    audio = batch["path"]["array"]
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)

    batch["input_values"] = inputs.input_values[0]
    batch["attention_mask"] = inputs.attention_mask[0]
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [22]:
# Apply the preprocess function to the dataset
encoded_dataset = dataset.map(preprocess_function, remove_columns=dataset.column_names, num_proc=1)

Map:   0%|          | 0/2884 [00:00<?, ? examples/s]c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 2884/2884 [00:43<00:00, 65.72 examples/s]


In [24]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./hubert-finetuned-catalan",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=3e-4,
    num_train_epochs=10,
    warmup_steps=500,
    save_total_limit=2,
    fp16=False,  # Turn true when GPU avilable
)

In [31]:
from typing import Any, Dict, List, Union
import torch
from transformers import Wav2Vec2Processor

class CustomDataCollatorCTCWithPadding:
    def __init__(self, processor: Wav2Vec2Processor, padding: Union[bool, str] = True):
        self.processor = processor
        self.padding = padding

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Split input values and labels
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Pad input values
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # Pad labels, replacing any padding token ids with -100 so they can be ignored during CTC loss computation
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # Replace padding with -100 to ignore them during loss calculation
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

# Use the custom data collator
data_collator = CustomDataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:

# Load metrics
from evaluate import load
wer_metric = load("wer")

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'add_model_info_to_custom_pipelines' from 'transformers.utils' (c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\__init__.py)

In [37]:
# Define a compute_metrics function
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = torch.argmax(torch.tensor(pred_logits), dim=-1)

    pred_str = processor.batch_decode(pred_ids)
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [38]:
# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,  # You can split the dataset for evaluation separately
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

In [39]:
# Fine-tune the model
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alexxsarda (iauab). Use `wandb login --relogin` to force relogin


  0%|          | 0/900 [00:00<?, ?it/s]c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
  0%|          | 1/900 [16:01<240:04:39, 961.38s/it]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\alexx\AppData\Local\Temp\ipykernel_21228\1814428710.py", line 2, in <module>
    trainer.train()
  File "c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer.py", line 1859, in train
    if train_bs_args != train_bs_state:
  File "c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer.py", line 2203, in _inner_training_loop
    if self.args.n_gpu > 1:
  File "c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer.py", line 3138, in training_step
    if isinstance(self.state.stateful_callbacks[cb_name], list):
  File "c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer.py", line 3161, in 